In [9]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=72a01542f30fa5b1e925b5f88354099939da190c62e1721b85938d63d3b57bcb
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=f1ebde492d0e8ac027891ff3e6aab05e9bbc2a57b8eec03589ecd91f6471796a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [11]:
fashion_mnist=keras.datasets.fashion_mnist

In [12]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [13]:
train_images=train_images/255.0
test_images=test_images/255.0

In [14]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [15]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv1filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv1kernel',values=[3,5]),
        activation='relu',
        input_shape =(28,28,1) 
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv2filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv2kernel',values=[3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense1units',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

  return model

In [16]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

In [17]:
tuner_search= RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion ')

In [18]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.9098333120346069

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 00h 02m 28s
INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        128064    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,103,626
Trainable params: 3,103,626
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1250 - accuracy: 0.9527 - val_loss: 0.2629 - val_accuracy: 0.9152
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0896 - accuracy: 0.9672 - val_loss: 0.2706 - val_accuracy: 0.9158
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0622 - accuracy: 0.9780 - val_loss: 0.3108 - val_accuracy: 0.9167
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0451 - accuracy: 0.9838 - val_loss: 0.3516 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0313 - accuracy: 0.9889 - val_loss: 0.4111 - val_accuracy: 0.9172
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0241 - accuracy: 0.9912 - val_loss: 0.4873 - val_accuracy: 0.9080
Epoch 10/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0218 - accuracy: 0.9927 - val_loss: 0.6146 - val_accuracy